<a href="https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab10/L10_Colab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 10.2 Machine Learning 3 - Real-time Sound Recognition for Classification

## Entire remote monitoring system


## 3.1 System configuration

Based on the sample code in the previous section, let's apply the CNN model to predict the operational state of the vacuum pump. To do this, we will use TensorFlow Lite. TensorFlow Lite has limited features but is faster and lighter than TensorFlow. When you install TensorFlow on Raspberry Pi and laptop, TensorFlow Lite module is automatically installed. Because it supports Keras interface, TensorFlow Lite is able to load and run the CNN model developed in the prelab using TensorFlow and Keras.

The best way to practice implementation of the trained model is to program yourself. Because of the limited time in the lab, however, the complete sample code ([lab10_sample2.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample2.py?raw=true)) is given. Perform Task 2.1 to interpret the model output as the appropriate label. Note that the CNN model (h5 format) must be in the same directory in which the sample code is.

<br>

### Task 2.1

Run [lab10_sample2.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample2.py?raw=true) on your Raspberry Pi or laptop see if the CNN model works well. The terminal or output shell looks like as Figure 5. **Note that if you develop the CNN model with different feature size and shape, you have to modify the sample code.** An algorithm to interpret the CNN model output is incomplete as part of the sample code below. Complete the algorithm so that appropriate label is printed out at the end. To be specific, if Y (prediction inference index) is 0, the prediction label must be "OFF" (given), if Y is 1, the prediction label must be "ON-airleaking", and so on.

* Hint: Use if, elif, else statement.
* Hint: Another way is to use dictionary data structure with indexing.

<br>

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Python.png?raw=tru" width="20">Python - Python3 (Last part of [lab10_sample2.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample2.py?raw=true))**

```
91         if Y == 0: # if Y is 0, do below
92             prediction_label = "OFF"
93         # complete your algorithm to take appropriate prediction_label string according to the CNN model result
94 
95         print('The pump is now {}.\n'.format(prediction_label))
96         
97         time.sleep(1) # pause for 1 second

```

---

<br>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig5.png?raw=true" width="80%">

*Figure 5 Terminal while running lab10_sample2.py*

In [ ]:
## copy and paste your entire algorithm from line number 91 (if Y == 0: # if Y is 0, do below) to the end



<br></br>

Please continue to [Lab 10.3 here](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab10/L10_Colab3.ipynb).
